In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'
if not os.path.exists(volume_dir):
    os.makedirs(volume_dir)

In [ ]:
###### reconstruct thumbnail volume  #######

stack = 'MD589'

downsample_factor = 32

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

n_sec = {'MD589': 445, 'MD594': 432}

dm = DataManager(stack=stack)

# volume = np.zeros((dm.image_height/downsample_factor, 
#                    dm.image_width/downsample_factor, 
#                    int(n_sec[stack]*z_xy_ratio_downsampled)), np.uint8)

volume = np.zeros((1500, 2000, int(n_sec[stack]*z_xy_ratio_downsampled)), np.uint8)

section_bs_begin, section_bs_end = section_range_lookup[stack]
print section_bs_begin, section_bs_end

for sec in range(section_bs_begin, section_bs_end+1):
    
    print sec
    
#     dm.set_slice(section=sec)

#     im = img_as_ubyte(rgb2gray(imread(os.environ['DATA_DIR'] + '/%(stack)s_thumbnail_aligned_cropped/%(stack)s_%(slice)04d_thumbnail_aligned_cropped.tif' \
#                         % {'stack': stack, 'slice': sec})))

    im = img_as_ubyte(imread(os.environ['DATA_DIR'] + '/%(stack)s_thumbnail_aligned_masked/%(stack)s_%(slice)04d_thumbnail_aligned_masked.png' \
        % {'stack': stack, 'slice': sec})) 
    
    volume[:,:, int(z_xy_ratio_downsampled*sec):int(z_xy_ratio_downsampled*(sec+1))] = im[..., np.newaxis]

In [ ]:
volume.shape

In [ ]:
# volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes'

volume[volume < 1e-6] = 0
xs, ys, zs = np.where(volume > 0)

xmin = xs.min()
xmax = xs.max()
ymin = ys.min()
ymax = ys.max()
zmin = zs.min()
zmax = zs.max()

print xmin, xmax
print ymin, ymax
print zmin, zmax

In [ ]:
volume = volume[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1]

In [ ]:
dm = DataManager(stack=stack)

theta_z = 0
theta_y = 0
theta_x = 0
tx = 0
ty = 0
tz = 0
skew_z = 0
skew_y = 0
skew_x = 0

# cos_z = np.cos(theta_z)
# sin_z = np.sin(theta_z)
# cos_y = np.cos(theta_y)
# sin_y = np.sin(theta_y)
# Rz = np.array([[cos_z, -sin_z, 0], [sin_z, cos_z, 0], [0,0,1]])
# Ry = np.array([[cos_y, 0, -sin_y], [0, 1, 0], [sin_y, 0, cos_y]])
# Rx = np.array([[1, 0, 0], [0, cos_y, -sin_y], [0, sin_y, cos_y]])
# R = np.dot(Rz, Ry, Rx)


In [ ]:
xs, ys, zs = np.meshgrid(range(volume.shape[0]), 
                         range(volume.shape[1]), 
                         range(volume.shape[2]))

In [ ]:
xys = np.c_[xs.flat, ys.flat, zs.flat]

In [ ]:
# manual affine correction
T = np.eye(3)
T[0,2] = .1 # + move right side down
T[2,0] = -.1 # + move bottom side left
# T[2,0] = .1

print T

In [ ]:
xyzs_new = np.dot(T, np.c_[xs.flat, ys.flat, zs.flat].T).astype(np.int).T

In [ ]:
valid = (xyzs_new[:,0] >= 0) & (xyzs_new[:,1] >= 0) & (xyzs_new[:,2] >= 0) &\
(xyzs_new[:,0] < volume.shape[0]) & (xyzs_new[:,1] < volume.shape[1]) & (xyzs_new[:,2] < volume.shape[2])

In [ ]:
# volume_new = np.zeros((dm.image_height/downsample_factor, 
#                    dm.image_width/downsample_factor, 
#                    int(n_sec[stack]*z_xy_ratio_downsampled)), np.uint8)

volume_new = np.zeros_like(volume, np.uint8)

volume_new[xyzs_new[valid, 0], xyzs_new[valid, 1], xyzs_new[valid, 2]] = \
volume[xys[valid, 0], xys[valid, 1], xys[valid, 2]]

In [ ]:
for y in range(0, volume.shape[1], 50):
    plt.imshow(volume_new[:, y, :], cmap=plt.cm.gray);
    plt.title('y = %d' % y);
    plt.show();

In [ ]:
for y in range(0, volume.shape[1], 10):
    plt.imshow(volume[:, y, :], cmap=plt.cm.gray);
    plt.title('y = %d' % y);
    plt.show();